# Pool - how it manages elements?

Contents:
    
* Overview each Pool element type:
 * Element overview
 * Element API (code snippet)
 * Tango element API

**Note**: The code snippets use the testing utilities, their API may change

## Motor

* [Motor overview](http://www.sardana-controls.org/en/latest/devel/overview/overview_motor.html#motor-overview)
* [Motor API reference](http://www.sardana-controls.org/en/latest/devel/api/api_motor.html#sardana-motor-api)
* [Tango Motor API reference](http://www.sardana-controls.org/en/latest/devel/api/sardana/tango/pool/Motor.html#sardana.tango.pool.Motor.Motor)

### Motor overview

* Interface was designed with physical motors in mind
* Interface is dynamic - only Position, State and Status attribute are mandatory
* Motor can be anything that can be changed e.g. power supply fits to a motor as well

### Motor API reference

* *position* and its relation with *dial position*, *offset* and *sign*
* *dial position* and its relation with *steps per unit*
* *state* and *limit switches*
* *velocity*, *acceleration*, *deceleration* and *base rate*
* *backlash* and *instability time*
* operations: *start move*, *stop* and *abort*

In [11]:
from sardana.pool.test import BasePoolTestCase

util = BasePoolTestCase()
util.setUp()
ctrl = util.createController("motctrl01",
                      "DummyMotorController",
                      "DummyMotorController.py")
mot = util.createMotorElement(ctrl, "mot01", 1)
print mot.get_state(), mot.get_position()
util.tearDown()

0 Position(
       name = Position
    manager = mot01
       read = 0 at 2017-07-27 14:19:53.444691
      write = -----)



### Tango Motor API

* All the Motor API are also available in Tango Motor API
* Extra interface
 * `DefinePosition` command
 * `MoveRelative` command \*
 * `SaveConfig` command \*
 * `SimulationMode` attribute \* 
 * `State` and `Limit_switches`events
 * `Position` events
 
\* Not implemented or not fully tested

## PseudoMotor

* [PseudoMotor overview](http://www.sardana-controls.org/en/latest/devel/overview/overview_pseudomotor.html)
* [PseudoMotor API reference](http://www.sardana-controls.org/en/latest/devel/api/api_pseudomotor.html#sardana-pseudomotor-api)
* [Tango PseudoMotor API reference](http://www.sardana-controls.org/en/latest/devel/api/sardana/tango/pool/PseudoMotor.html#sardana.tango.pool.PseudoMotor.PseudoMotor)

### Motor overview
* Interface was designed with physical motors in mind
* Interface is dynamic - only Position, State and Status attribute are mandatory
* Motor can be anything that can be changed e.g. power supply fits to a motor as well